##### Copyright 2018 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 텍스트 로드

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

이 튜토리얼은 `tf.data.TextLineDataset`를 사용하여 텍스트 파일에서 예제를 로드하는 방법에 대한 예제를 제공합니다. `TextLineDataset`는 텍스트 파일에서 데이터세트를 작성하도록 설계되었으며, 이때 각 예제는 원본 파일의 텍스트 줄입니다. 주로 줄 기반의 텍스트 데이터(예: 시 또는 오류 로그)에 유용합니다.

이 튜토리얼에서는 같은 작품에 대한 3가지 다른 영어 번역본이 있는 Homer's Illiad를 사용하여 한 줄의 텍스트로 번역기를 식별하는 모델을 훈련합니다.

## 설정

In [ ]:
!pip install -q tfds-nightly

In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

3가지 번역본은 다음과 같습니다.

- [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) — [텍스트](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)

- [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) — [텍스트](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)

- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) — [텍스트](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

이 튜토리얼에 사용된 텍스트 파일은 문서 헤더와 바닥 글, 줄 번호, 챕터 제목 등을 제거하는 일반적인 전처리 작업을 거쳤습니다. 이 가볍게 손질한 파일을 로컬로 다운로드합니다.

In [ ]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)
  
parent_dir = os.path.dirname(text_dir)

parent_dir

## 데이터세트에 텍스트 로드하기

파일을 반복하면서 각 파일을 자체 데이터세트에 로드합니다.

각 예제에 개별적으로 레이블을 지정해야 하므로 `tf.data.Dataset.map`을 사용하여 각 예제에 labeler 함수를 적용합니다. 이렇게 하면 데이터세트의 모든 예제를 반복하여 (`example, label`) 쌍을 반환합니다.

In [ ]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

레이블이 지정된 데이터세트를 단일 데이터세트로 결합하고 섞습니다.


In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [ ]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

`tf.data.Dataset.take` 및 `print`를 사용하여 `(example, label)` 쌍의 형태를 확인할 수 있습니다. `numpy` 속성은 각 텐서의 값을 보여줍니다.

In [ ]:
for ex in all_labeled_data.take(5):
  print(ex)

## 텍스트 줄을 숫자로 인코딩하기

머신러닝 모델은 단어가 아닌 숫자에 대해 동작하므로 문자열 값을 숫자 목록으로 변환해야 합니다. 그렇게 하려면 각 고유한 단어를 고유한 정수에 매핑합니다.

### 어휘 만들기

먼저, 텍스트를 개별 고유 단어 모음으로 토큰화하여 어휘를 작성합니다. TensorFlow와 Python에서 이를 수행하는 몇 가지 방법이 있습니다. 이 튜토리얼에서 사용한 방법은 다음과 같습니다.

1. 각 예제의 `numpy` 값을 반복합니다.
2. `tfds.features.text.Tokenizer`를 사용하여 토큰으로 분할합니다.
3. 이들 토큰을 Python 세트로 수집하여 중복을 제거합니다.
4. 나중에 사용할 수 있도록 어휘의 크기를 구합니다.

In [ ]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

### 인코딩 예제

`vocabulary_set`를 `tfds.features.text.TokenTextEncoder`에 전달하여 encoder를 작성합니다. encoder의 `encode` 메서드는 텍스트 문자열을 받아서 정수 목록을 반환합니다.

In [ ]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

한 줄로 시도하여 출력 결과를 확인할 수 있습니다.

In [ ]:
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

In [ ]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

이제 `tf.py_function`으로 래핑하여 데이터세트의 `map` 메서드에 전달하여 데이터세트에 대해 encoder를 실행합니다.

In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

`Dataset.map`을 사용하여 이 함수를 데이터세트의 각 요소에 적용하려고 합니다. `Dataset.map`은 그래프 모드에서 실행됩니다.

- 그래프 텐서에는 값이 없습니다.
- 그래프 모드에서는 TensorFlow Ops 및 함수만 사용할 수 있습니다.

따라서 이 함수를 직접 `.map`할 수 없습니다. `tf.py_function`로 래핑해야 합니다. `tf.py_function`은 래핑된 파이썬 함수에 정규 텐서(값과 `.numpy()` 메서드를 사용하여 액세스)를 전달합니다.

In [ ]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

## 데이터세트를 테스트 및 훈련 배치로 분할하기

`tf.data.Dataset.take` 및 `tf.data.Dataset.skip`을 사용하여 작은 테스트 데이터세트와 더 큰 훈련 세트를 작성합니다.

모델에 전달되기 전에 데이터세트를 배치 처리되어야 합니다. 일반적으로, 배치 내부의 예제는 크기와 형상이 같아야 합니다. 그러나 이 데이터세트의 예제는 모두 같은 크기가 아닙니다. 각 텍스트 줄의 단어 수는 다릅니다. 따라서 `batch` 대신 `tf.data.Dataset.padded_batch`를 사용하여 예제를 같은 크기로 채워야 합니다.

In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

이제 `test_data` 및 `train_data`는 (`example, label`) 쌍의 모음이 아니라 배치의 모음입니다. 각 배치는 배열로 표시되는 한 쌍의 (*많은 예제*, *많은 레이블*)입니다.

다음과 같습니다.

In [ ]:
sample_text, sample_labels = next(iter(test_data))

sample_text[0], sample_labels[0]

새로운 토큰 인코딩(패딩에 사용되는 0)을 도입했기 때문에 어휘 크기가 1씩 증가했습니다.

In [ ]:
vocab_size += 1

## 모델 빌드하기


In [ ]:
model = tf.keras.Sequential()

첫 번째 레이어는 정수 표현을 고밀도 벡터 임베딩으로 변환합니다. [단어 임베딩 튜토리얼](../text/word_embeddings.ipynb) 또는 자세한 내용을 참조하세요. 

In [ ]:
model.add(tf.keras.layers.Embedding(vocab_size, 64))

다음 레어어는 [Long Short-Term Memory](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) 레이어로, 모델이 문맥에서 단어를 다른 단어와 함께 이해할 수 있도록 합니다. LSTM의 양방향 래퍼는 모델 이전 및 이후에 오는 데이터 포인트와 관련하여 데이터 포인트를 학습하는 데 도움이 됩니다.

In [ ]:
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

최종적으로, 일련의 하나 이상의 조밀하게 연결된 레이어를 갖게 되며 마지막 레이어는 출력 레이어입니다. 출력 레이어는 모든 레이블에 대한 확률을 생성합니다. 확률이 가장 높은 레이블은 예제 레이블의 모델 예측입니다.

In [ ]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))

끝으로, 모델을 컴파일합니다. softmax 분류 모델의 경우, 손실 함수로 `sparse_categorical_crossentropy`를 사용합니다. 다른 옵티마이저를 사용해 볼 수 있지만, `adam`은 매우 일반적입니다.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 모델 훈련하기

이 데이터에 대해 실행되는 이 모델은 적절한 결과(약 83%)를 생성합니다.

In [ ]:
model.fit(train_data, epochs=3, validation_data=test_data)

In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))